In [ ]:
import random

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical



## Load dataset

In [ ]:
(x_train, y_train),(x_test, y_test) = keras.datasets.mnist.load_data()

img_count = x_train.shape[0]
img_x_dim = x_train.shape[1]
img_y_dim = x_train.shape[2]

## Confirm handwritten image vs truth

In [ ]:
i = random.randrange(img_count)
print(f"The handwritten sample #{i} is a '{y_train[i]}'")
plt.imshow(x_train[i], cmap='gray_r', vmin = 0, vmax = 255);

## Pre-processing

#### Reshape input data
Input data is N X*Y sized images in the form of [N, X, Y]. So reshape into rows.

In [ ]:
xx_train = np.reshape(x_train, (x_train.shape[0], img_x_dim*img_y_dim))
xx_test = np.reshape(x_test, (x_test.shape[0], img_x_dim*img_y_dim))


In [ ]:
# print random row in original and reshaped data to confirm expected reshape
j = random.randrange(28)
print(f"Row: {j}")
print(f" - Original = {x_train[i][j]}")
print(f" - Reshaped = {xx_train[i][j*img_y_dim:(j+1)*img_y_dim]}")

#### Encode truth data
Truth data needs to be one-hot encoded

In [ ]:
import numpy as np

num_classes = 10
yy_train = to_categorical(y_train, num_classes)
yy_test = to_categorical(y_test, num_classes)


print(y_train[i:i+5])
print('  -------------------')
print(f"#{[0,1,2,3,4,5,6,7,8,9]}")
print(yy_train[0:9])

## Define model

In [ ]:
def mnist_model(x, y, epochs, batch_size, model_layout):
    
    # create train & val sets
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.1, shuffle=True)
    
    # define model
    model = Sequential()
    
    # first hidden layer
    model.add(Dense(model_layout[0], input_dim=img_x_dim*img_y_dim, activation='relu'))

    # remaining hidden layers
    depth = len(model_layout)
    for width in model_layout[1:]:
        model.add(Dense(width, activation='relu'))

    # output layer
    model.add(Dense(10, activation='softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())

    # train (fit) the model to the dataset
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=1)

    # plot loss during training
    plt.title('Loss')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    return model


In [ ]:
my_model = mnist_model(xx_train, yy_train, 15, 128, [784, 784/4, 784/16])

## Evaluate model

In [ ]:
# make probability predictions with the model
predictions = my_model.predict(xx_test)
pred_labels = np.argmax(predictions, axis=1)

# evaluate the keras model
_, train_mse = my_model.evaluate(xx_train, yy_train, verbose=0)
_, test_mse = my_model.evaluate(xx_test, yy_test, verbose=0)

print('Train MSE:  %.4f' % train_mse)
print('Test MSE:  %.4f' % test_mse)

# save off the wrong predictions
wrong_predictions = x_test[pred_labels != y_test]
wrong_labels = pred_labels[pred_labels != y_test]
right_labels = y_test[pred_labels != y_test]

# Characterize amount of wrongness
percent_wrong = wrong_predictions.shape[0]/x_test.shape[0]*100
print(f"Number Wrongly Predicted:  {wrong_predictions.shape[0]}")
print(f"Percent Wrong Predictions: {percent_wrong:.2f}%")

#### Review wrong predictions

In [ ]:
random_wrong = random.randrange(wrong_predictions.shape[0])
print(f"Handwritten sample #{random_wrong}:")
print(f" - Predicted: '{wrong_labels[random_wrong]}'")
print(f" - Actual:     '{right_labels[random_wrong]}'")
plt.imshow(wrong_predictions[random_wrong], cmap='gray_r', vmin = 0, vmax = 255);

## Visualize Model Layers

In [ ]:
print(my_model.layers)
# cw1 = np.array(my_model.layers[1].get_weights())
# print(cw1.shape) # 2 weight, 1 weight, 1 bias
# print(cw1[0].shape) # 3 channels, 3 by 3 kernels, 32 filters
# print(cw1[1].shape) # 32 biases

# cw1 = np.array(my_model.layers[2].get_weights())
# print(cw1.shape) # this is just a flatten operations, so no weights

# cw1 = np.array(my_model.layers[3].get_weights())
# print(cw1.shape) # 2 -> 1 weight, 1 bias
# print(cw1[0].shape) # 28800 inputs, 5 outputs, 28800 by 5 weight matrix
# print(cw1[1].shape) # 5 biases

In [ ]:
from keras import backend as K

for w in my_model.trainable_weights:
#     print(w)
    print(K.eval(w).shape)

In [ ]:
plt.imshow(K.eval(w)[0][0], cmap='gray_r', vmin = 0, vmax = 255);

In [ ]:
print(my_model.get_config)

In [ ]:
# first layer
a = K.eval(my_model.trainable_weights[0][2])
print(a.shape)
aa = np.reshape(a, (28, 28))
plt.imshow(aa);

In [ ]:
# second layer
b = K.eval(my_model.trainable_weights[3])
print(b.shape)
bb = np.reshape(b, (14, 14))
plt.imshow(bb);

In [ ]:
# third layer
c = K.eval(my_model.trainable_weights[5])
print(c.shape)
cc = np.reshape(c, (7, 7))
plt.imshow(cc);